# Math.NET Symbolics Examples

This is an example F# Notebook showing how [Math.NET Symbolics](http://symbolics.mathdotnet.com/) can be used interactively.

To get started, we need to load Math.NET Symbolics using Paket, open namespaces and make sure expression are presented nicely in LaTeX:

In [40]:
// Use the Sylvester.tf NuGet packages inlcuding the native Windows depedency
#load "Paket.fsx"
Paket.Package["MathNet.Numerics"; "MathNet.Symbolics"] 
#load "Paket.Generated.Refs.fsx"

open MathNet.Numerics
open MathNet.Symbolics

open Operators

App.AddDisplayPrinter (fun (x:Expression) -> { ContentType = "text/latex"; Data = "$$" + LaTeX.format x + "$$" })

Then let's create a few symbols we'd like to use in our algebraic expressions:

In [75]:
let x = symbol "x"
let y = symbol "y"
let z = symbol "z"
let a = symbol "a"
let b = symbol "b"
let c = symbol "c"
let d = symbol "d"
let e = symbol "e"
let f = symbol "f"
let H = symbol "H"
let L = symbol "L"

In [79]:
L+H+L+H - (L+H) + 4. * L - 5. * H |> Polynomial.coefficients (5 * H)

[|Sum
    [Product [Approximation (Real -4.0); Identifier (Symbol "H")];
     Product [Approximation (Real 5.0); Identifier (Symbol "L")]]|]

Using these symbols we can then use normal F# notation to construct algebraic expressions. Expressions are automatically simplifed.

In [42]:
(a/b/(c*a))*(c*d/a)/d

Product
  [Power (Identifier (Symbol "a"),Number -1N);
   Power (Identifier (Symbol "b"),Number -1N)]

In [43]:
(a/b)**c

Power
  (Product [Identifier (Symbol "a"); Power (Identifier (Symbol "b"),Number -1N)],
   Identifier (Symbol "c"))

You can use `Q` to force a number to become a symbolic rational number expression, e.g. if an operator is used with only numeric operands. Otherwise F# would evaluate it, and e.g. perform an integer division as below. If at least one operand is an expression, the result will be an expression as well.

In [44]:
3/2

1

In [45]:
3Q/2

Number 3/2N

In [46]:
(1Q/2)*sin(x)

Product [Number 1/2N; Function (Sin,Identifier (Symbol "x"))]

### Algebraic Functions

Symbolics provides a set of F# modules with functions to manipulate expressions.

In [47]:
Exponential.expand (exp(2*x+y))

Product
  [Power (Function (Exp,Identifier (Symbol "x")),Number 2N);
   Function (Exp,Identifier (Symbol "y"))]

In [48]:
Trigonometric.expand (sin(a+x))

Sum
  [Product
     [Function (Sin,Identifier (Symbol "x"));
      Function (Cos,Identifier (Symbol "a"))];
   Product
     [Function (Sin,Identifier (Symbol "a"));
      Function (Cos,Identifier (Symbol "x"))]]

In [49]:
Trigonometric.contract (sin(a)*sin(b))

Sum
  [Product
     [Number -1/2N;
      Function (Cos,Sum [Identifier (Symbol "a"); Identifier (Symbol "b")])];
   Product
     [Number 1/2N;
      Function
        (Cos,
         Sum
           [Identifier (Symbol "a");
            Product [Number -1N; Identifier (Symbol "b")]])]]

In [50]:
Trigonometric.contract (cos(x)**4)

Sum
  [Number 3/8N;
   Product
     [Number 1/2N; Function (Cos,Product [Number 2N; Identifier (Symbol "x")])];
   Product
     [Number 1/8N; Function (Cos,Product [Number 4N; Identifier (Symbol "x")])]]

The functions are grouped by the context we interpret the expression in. Functions in `Polynomial` assume the expression is a single- or multivariate polynomial.

In [51]:
Polynomial.quot x (x**3 - 2*x**2 - 4) (x-3)

Sum
  [Number 3N; Identifier (Symbol "x"); Power (Identifier (Symbol "x"),Number 2N)]

In [52]:
(a/b)+(c/d)+(e/f)

Sum
  [Product [Identifier (Symbol "a"); Power (Identifier (Symbol "b"),Number -1N)];
   Product [Identifier (Symbol "c"); Power (Identifier (Symbol "d"),Number -1N)];
   Product [Identifier (Symbol "e"); Power (Identifier (Symbol "f"),Number -1N)]]

In [53]:
Rational.expand (a/b + c/d + e/f)

Product
  [Power (Identifier (Symbol "b"),Number -1N);
   Power (Identifier (Symbol "d"),Number -1N);
   Power (Identifier (Symbol "f"),Number -1N);
   Sum
     [Product
        [Identifier (Symbol "b"); Identifier (Symbol "d");
         Identifier (Symbol "e")];
      Product
        [Identifier (Symbol "b"); Identifier (Symbol "c");
         Identifier (Symbol "f")];
      Product
        [Identifier (Symbol "a"); Identifier (Symbol "d");
         Identifier (Symbol "f")]]]

In [54]:
Rational.rationalize (1+1/(1+1/x))

Product
  [Power (Sum [Number 1N; Identifier (Symbol "x")],Number -1N);
   Sum [Number 1N; Product [Number 2N; Identifier (Symbol "x")]]]

In [55]:
Polynomial.gcd x (x**7 - 4*x**5 - x**2 + 4) (x**5 - 4*x**3 - x**2 + 4)

Sum
  [Number 4N; Product [Number -4N; Identifier (Symbol "x")];
   Product [Number -1N; Power (Identifier (Symbol "x"),Number 2N)];
   Power (Identifier (Symbol "x"),Number 3N)]

In [56]:
Polynomial.polynomialExpansion x y (x**5 + 11*x**4 + 51*x**3 + 124*x**2 + 159*x + 86) (x**2 + 4*x + 5)

Sum
  [Number 1N; Identifier (Symbol "x");
   Product [Sum [Number 2N; Identifier (Symbol "x")]; Identifier (Symbol "y")];
   Product
     [Sum [Number 3N; Identifier (Symbol "x")];
      Power (Identifier (Symbol "y"),Number 2N)]]

In [57]:
let expr = (8*a*x + 6*a*x**2)/(4*x*a) |> Rational.expand
expr

Sum [Number 2N; Product [Number 3/2N; Identifier (Symbol "x")]]

In [58]:
2*expr |> Algebraic.expand

Sum [Number 4N; Product [Number 3N; Identifier (Symbol "x")]]

In [59]:
Approximate.approximate expr

Sum
  [Approximation (Real 2.0);
   Product [Approximation (Real 1.5); Identifier (Symbol "x")]]

In [60]:
Rational.simplify x ((x**2-1)/(x+1))

Sum [Number -1N; Identifier (Symbol "x")]

In [61]:
Rational.simplify x (1/(1+1/(x+1)) + 2/(x+2))

Product
  [Power (Sum [Number 2N; Identifier (Symbol "x")],Number -1N);
   Sum [Number 3N; Identifier (Symbol "x")]]

In [62]:
(x**8 + 6*x**6 + 12*x**4 + 8*x**2)

Sum
  [Product [Number 8N; Power (Identifier (Symbol "x"),Number 2N)];
   Product [Number 12N; Power (Identifier (Symbol "x"),Number 4N)];
   Product [Number 6N; Power (Identifier (Symbol "x"),Number 6N)];
   Power (Identifier (Symbol "x"),Number 8N)]

In [63]:
(x**8 + 6*x**6 + 12*x**4 + 8*x**2) |> Polynomial.factorSquareFree x 

Product
  [Power (Identifier (Symbol "x"),Number 2N);
   Power (Sum [Number 2N; Power (Identifier (Symbol "x"),Number 2N)],Number 3N)]

### Calculus

In [64]:
Calculus.differentiate x (a*x**2 + b*x + c)

Sum
  [Identifier (Symbol "b");
   Product [Number 2N; Identifier (Symbol "a"); Identifier (Symbol "x")]]

In [65]:
(x**3 - 12*x**2 - c) |> Calculus.tangentLine x 1Q

Sum
  [Number 10N; Product [Number -1N; Identifier (Symbol "c")];
   Product [Number -21N; Identifier (Symbol "x")]]

In [66]:
1/z |> Calculus.normalLine z 3Q

Sum [Number -80/3N; Product [Number 9N; Identifier (Symbol "z")]]

### Working with Expressions

Of course we can also combine these functions to write more powerful function on top of them:

In [67]:
let taylor (k:int) symbol a x =
    let rec impl n factorial sum dxn =
        if n = k then sum else
        let dxn_a = dxn |> Structure.substitute symbol a
        let dxn'  = dxn |> Calculus.differentiate symbol
        impl (n+1) (factorial*(n+1)) (sum + dxn_a/factorial*(symbol-a)**n) dxn'
    impl 0 1 zero x |> Algebraic.expand

taylor 4 x 0Q (sin(x)+cos(x))

Sum
  [Number 1N; Identifier (Symbol "x");
   Product [Number -1/2N; Power (Identifier (Symbol "x"),Number 2N)];
   Product [Number -1/6N; Power (Identifier (Symbol "x"),Number 3N)]]

In [68]:
let tangent symbol a x = taylor 2 symbol a x

tangent x 0Q (sin(x)+cos(x))

Sum [Number 1N; Identifier (Symbol "x")]

### Simplistic solver

In [69]:
let solve x a b =

    let a' = Rational.simplify x a
    let an = Rational.numerator a'
    let ad = Rational.denominator a'
    
    let b' = Rational.simplify x b
    let bn = Rational.numerator b'
    let bd = Rational.denominator b'
    
    let expr = an*bd - bn*ad |> Algebraic.expand
    
    if Polynomial.isPolynomial x expr then
        match Polynomial.coefficients x expr with
        | [||] -> Undefined
        | [| a |] -> x
        | [| a; b |] -> -a/b |> Rational.simplify x
        | _ -> failwith "higher polynomials not supported"
    else failwith "only general polynomial expressions supported for now"
    
// x/(x+5) = 7 --> x=?
solve x (x/(x + 5)) (number 7)

Number -35/6N

In [70]:
// sin(a)+x*cos(b) = c --> x=?
solve x (sin(a)+x*cos(b)) c

Product
  [Sum
     [Identifier (Symbol "c");
      Product [Number -1N; Function (Sin,Identifier (Symbol "a"))]];
   Power (Function (Cos,Identifier (Symbol "b")),Number -1N)]

### Formatting and Parsing Expressions

In [71]:
Infix.parseOrUndefined "(a/b)*sin(x)"

Product
  [Identifier (Symbol "a"); Power (Identifier (Symbol "b"),Number -1N);
   Function (Sin,Identifier (Symbol "x"))]

In [72]:
Quotations.parse <@ fun x y -> -x*y/3 @>

Product [Number -1/3N; Identifier (Symbol "x"); Identifier (Symbol "y")]

In [73]:
MathML.formatContentStrict (x**(1Q/3))

"<apply>
  <csymbol cd="arith1">root</csymbol>
  <ci>x</ci>
  <cn>3</cn>
</apply>"

In [74]:
MathML.parse """<csymbol cd="nums1">pi</csymbol>"""

Constant Pi